<a href="https://colab.research.google.com/github/Adlucent/ga4-return-prediction/blob/main/6_Scale_Data_and_Customer_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6. Scale Data and Customer Segmentation

In [ ]:
################################################################################
######################### CHANGE BQ PROJECT NAME BELOW #########################
################################################################################

project_name = 'adl-analytics' #add proj name
region = "US"  # GCP project region
table_name = 'adl-analytics.return_prediction_ga4.step_5_merged_final'
customer_segment_table = 'adl-analytics.public_sample_data.synthetic_returns_ga4'

In [ ]:
# If your notebook does not have pandas_gbq you can install it here:
# ! pip install pandas_gbq

In [ ]:
# Google credentials
from google.colab import auth
auth.authenticate_user()

# BigQuery Magics
'''BigQuery magics are used to run BigQuery SQL queries in a python environment.
These queries can also be run in the BigQuery UI '''

from google.cloud import bigquery
from google.cloud.bigquery import magics, Client, QueryJobConfig

magics.context.project = project_name #update project name
client = bigquery.Client(project=magics.context.project)

# Interface between Jupyter and BigQuery
import pandas_gbq

# data processing libraries + ML tools
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

# suppress notebook warnings
import warnings
warnings.filterwarnings('ignore')

# dataframe formatting
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 50)
pd.set_option("display.float_format", lambda x: "%.3f" % x)

# Load data

In [ ]:
sql = f"""
SELECT *
FROM `{table_name}`;
"""
data = pandas_gbq.read_gbq(sql, project_id=project_name, location=region, use_bqstorage_api=True)

Downloading: 100%|██████████|


In [ ]:
# create a copy of the imported data to avoid re-importing if we need to revert to original table
df = data.copy()

In [ ]:
df.head()

,user_pseudo_id,ecommerce_transaction_id,transaction_ga_session_number,user_ltv_revenue,ecommerce_tax_value_in_usd,sum_item_price_in_usd,sum_item_promotions,sum_item_quantity,sum_event_name_begin_checkout,sum_event_name_select_item,sum_device_category_mobile,sum_device_mobile_brand_name_Google,sum_device_mobile_brand_name_Huawei,sum_device_mobile_brand_name_Microsoft,sum_device_mobile_brand_name_Samsung,sum_device_mobile_brand_name_Xiaomi,sum_device_mobile_model_name_ChromeBook,sum_device_mobile_model_name_Safari,sum_device_mobile_model_name_iPad,sum_device_mobile_model_name_iPhone,sum_device_web_info_browser_AndroidWebview,sum_device_web_info_browser_Firefox,sum_device_web_info_browser_Safari,sum_geo_country_Canada,sum_geo_country_France,sum_geo_country_India,sum_geo_country_Other,sum_geo_country_Spain,sum_geo_country_UnitedKingdom,sum_traffic_source_medium_Other,sum_traffic_source_medium_cpc,sum_traffic_source_medium_organic,sum_traffic_source_medium_referral,sum_event_params_parent_page_CampusCollection,sum_event_params_parent_page_CheckoutConfirmation,sum_event_params_parent_page_EcoFriendly,sum_event_params_parent_page_Home,sum_event_params_parent_page_Lifestyle,sum_event_params_parent_page_New,sum_event_params_parent_page_Other,sum_event_params_parent_page_PaymentMethod,sum_event_params_parent_page_Sale,sum_event_params_parent_page_ShoppingCart,sum_event_params_child_page_Bags,sum_event_params_child_page_Google,sum_event_params_child_page_Hats,sum_event_params_child_page_Kids,sum_event_params_child_page_Notebooks,sum_event_params_child_page_Other,sum_event_params_child_page_Socks,sum_event_params_child_page_Stickers,sum_event_params_child_page_Womens,sum_event_params_child_page_Writing,sum_event_params_child_page_YouTube,sum_item_parent_category_Other,sum_item_parent_category_Stationery,sum_item_child_category_Drinkware,sum_item_child_category_Google,sum_item_child_category_MensUnisex,sum_item_child_category_SmallGoods,sum_item_child_subcategory_Backpacks,sum_item_child_subcategory_ElectronicsAccessories,sum_item_child_subcategory_Infant,sum_item_child_subcategory_MensTShirts,sum_item_child_subcategory_MugsTumblers,sum_item_child_subcategory_Other,sum_item_child_subcategory_WaterBottles,max_event_params_engagement_time_msec,max_item_promotions,avg_event_params_engagement_time_msec,avg_item_promotions,avg_item_quantity,avg_item_revenue_in_usd,pre_max_event_params_ga_session_number,pre_avg_event_params_engagement_time_msec,pre_sum_event_name_scroll,pre_sum_event_name_select_item,pre_sum_device_mobile_brand_name_Google,pre_sum_device_mobile_brand_name_Samsung,pre_sum_device_mobile_brand_name_Xiaomi,pre_sum_device_mobile_model_name_ChromeBook,pre_sum_device_mobile_model_name_Safari,pre_sum_device_mobile_model_name_iPad,pre_sum_device_web_info_browser_AndroidWebview,pre_sum_device_web_info_browser_Edge,pre_sum_device_web_info_browser_Firefox,pre_sum_geo_country_Canada,pre_sum_geo_country_India,pre_sum_geo_country_Other,pre_sum_geo_country_Spain,pre_sum_geo_country_UnitedKingdom,pre_sum_traffic_source_medium_Other,pre_sum_traffic_source_medium_cpc,pre_sum_traffic_source_medium_organic,pre_sum_traffic_source_medium_referral,pre_sum_event_params_parent_page_CampusCollection,pre_sum_event_params_parent_page_CheckoutConfirmation,pre_sum_event_params_parent_page_EcoFriendly,pre_sum_event_params_parent_page_Home,pre_sum_event_params_parent_page_PaymentMethod,pre_sum_event_params_parent_page_Sale,pre_sum_event_params_child_page_Hats,pre_sum_event_params_child_page_Socks,pre_sum_event_params_child_page_Stickers,pre_sum_event_params_child_page_Womens,pre_sum_event_params_child_page_YouTube,pre_sum_item_parent_category_New,pre_sum_item_child_category_Bags,pre_sum_item_child_category_Drinkware,pre_sum_item_child_category_Kids,pre_sum_item_child_category_MensUnisex,pre_sum_item_child_subcategory_Backpacks,pre_sum_item_child_subcategory_ElectronicsAccessories,pre_sum_item_child_subcategory_WaterBottles,days_first_session_to_transaction,his

In [ ]:
df.shape

(4466, 185)

In [ ]:
# check for missing values
df.isna().sum().sum()

0

# Customer segmentation
As an additional feature to the model, it may be useful to cluster customers based on their history. <br>
This can be achieved using a clustering algorithm, such as KMeans, on your customer history data. <br>
In this case, we have already performed customer segmentation outside and will pull in the results here. <br>
**Add more verbiage here/how to**

In [ ]:
sql = f"""
SELECT DISTINCT user_pseudo_id, customer_segment
FROM `{customer_segment_table}`;
"""
segments = pandas_gbq.read_gbq(sql, project_id=project_name, location=region, use_bqstorage_api=True)

Downloading: 100%|██████████|


In [ ]:
customer_segments = segments.copy()

In [ ]:
customer_segments.head()

,user_pseudo_id,customer_segment
0,4268698.4893393516,9
1,22300841.9909876299,0
2,4853857.3181516287,0
3,50121199.4323445332,9
4,14107430.8136146352,9


In [ ]:
# add customer segment to our existing dataset
df = pd.merge(df, customer_segments, how='left', on='user_pseudo_id')
df.head()

,user_pseudo_id,ecommerce_transaction_id,transaction_ga_session_number,user_ltv_revenue,ecommerce_tax_value_in_usd,sum_item_price_in_usd,sum_item_promotions,sum_item_quantity,sum_event_name_begin_checkout,sum_event_name_select_item,sum_device_category_mobile,sum_device_mobile_brand_name_Google,sum_device_mobile_brand_name_Huawei,sum_device_mobile_brand_name_Microsoft,sum_device_mobile_brand_name_Samsung,sum_device_mobile_brand_name_Xiaomi,sum_device_mobile_model_name_ChromeBook,sum_device_mobile_model_name_Safari,sum_device_mobile_model_name_iPad,sum_device_mobile_model_name_iPhone,sum_device_web_info_browser_AndroidWebview,sum_device_web_info_browser_Firefox,sum_device_web_info_browser_Safari,sum_geo_country_Canada,sum_geo_country_France,sum_geo_country_India,sum_geo_country_Other,sum_geo_country_Spain,sum_geo_country_UnitedKingdom,sum_traffic_source_medium_Other,sum_traffic_source_medium_cpc,sum_traffic_source_medium_organic,sum_traffic_source_medium_referral,sum_event_params_parent_page_CampusCollection,sum_event_params_parent_page_CheckoutConfirmation,sum_event_params_parent_page_EcoFriendly,sum_event_params_parent_page_Home,sum_event_params_parent_page_Lifestyle,sum_event_params_parent_page_New,sum_event_params_parent_page_Other,sum_event_params_parent_page_PaymentMethod,sum_event_params_parent_page_Sale,sum_event_params_parent_page_ShoppingCart,sum_event_params_child_page_Bags,sum_event_params_child_page_Google,sum_event_params_child_page_Hats,sum_event_params_child_page_Kids,sum_event_params_child_page_Notebooks,sum_event_params_child_page_Other,sum_event_params_child_page_Socks,sum_event_params_child_page_Stickers,sum_event_params_child_page_Womens,sum_event_params_child_page_Writing,sum_event_params_child_page_YouTube,sum_item_parent_category_Other,sum_item_parent_category_Stationery,sum_item_child_category_Drinkware,sum_item_child_category_Google,sum_item_child_category_MensUnisex,sum_item_child_category_SmallGoods,sum_item_child_subcategory_Backpacks,sum_item_child_subcategory_ElectronicsAccessories,sum_item_child_subcategory_Infant,sum_item_child_subcategory_MensTShirts,sum_item_child_subcategory_MugsTumblers,sum_item_child_subcategory_Other,sum_item_child_subcategory_WaterBottles,max_event_params_engagement_time_msec,max_item_promotions,avg_event_params_engagement_time_msec,avg_item_promotions,avg_item_quantity,avg_item_revenue_in_usd,pre_max_event_params_ga_session_number,pre_avg_event_params_engagement_time_msec,pre_sum_event_name_scroll,pre_sum_event_name_select_item,pre_sum_device_mobile_brand_name_Google,pre_sum_device_mobile_brand_name_Samsung,pre_sum_device_mobile_brand_name_Xiaomi,pre_sum_device_mobile_model_name_ChromeBook,pre_sum_device_mobile_model_name_Safari,pre_sum_device_mobile_model_name_iPad,pre_sum_device_web_info_browser_AndroidWebview,pre_sum_device_web_info_browser_Edge,pre_sum_device_web_info_browser_Firefox,pre_sum_geo_country_Canada,pre_sum_geo_country_India,pre_sum_geo_country_Other,pre_sum_geo_country_Spain,pre_sum_geo_country_UnitedKingdom,pre_sum_traffic_source_medium_Other,pre_sum_traffic_source_medium_cpc,pre_sum_traffic_source_medium_organic,pre_sum_traffic_source_medium_referral,pre_sum_event_params_parent_page_CampusCollection,pre_sum_event_params_parent_page_CheckoutConfirmation,pre_sum_event_params_parent_page_EcoFriendly,pre_sum_event_params_parent_page_Home,pre_sum_event_params_parent_page_PaymentMethod,pre_sum_event_params_parent_page_Sale,pre_sum_event_params_child_page_Hats,pre_sum_event_params_child_page_Socks,pre_sum_event_params_child_page_Stickers,pre_sum_event_params_child_page_Womens,pre_sum_event_params_child_page_YouTube,pre_sum_item_parent_category_New,pre_sum_item_child_category_Bags,pre_sum_item_child_category_Drinkware,pre_sum_item_child_category_Kids,pre_sum_item_child_category_MensUnisex,pre_sum_item_child_subcategory_Backpacks,pre_sum_item_child_subcategory_ElectronicsAccessories,pre_sum_item_child_subcategory_WaterBottles,days_first_session_to_transaction,his

# Scale data
In order to make our algorithm converge faster and to make all features on the same relative scale, we'll perform a standard scaler transformation on the data.

In [ ]:
# select only features, not ID columns, target or split column
X = df[list(df.columns[2:-3]) + ['customer_segment']]
X.shape

(4466, 182)

In [ ]:
scaler = StandardScaler()
scaler.fit(X[df['split'] == 'TRAIN'])
X_scaled = scaler.transform(X)

In [ ]:
df_scaled = pd.DataFrame(X_scaled, columns=list(df.columns[2:-3]) + ['customer_segment'])
df_scaled = pd.concat([df[['user_pseudo_id', 'ecommerce_transaction_id']],
                       df_scaled,
                       df[['split', 'ecommerce_refund_value_in_usd']]],
                      axis=1)
df_scaled.head()

,user_pseudo_id,ecommerce_transaction_id,transaction_ga_session_number,user_ltv_revenue,ecommerce_tax_value_in_usd,sum_item_price_in_usd,sum_item_promotions,sum_item_quantity,sum_event_name_begin_checkout,sum_event_name_select_item,sum_device_category_mobile,sum_device_mobile_brand_name_Google,sum_device_mobile_brand_name_Huawei,sum_device_mobile_brand_name_Microsoft,sum_device_mobile_brand_name_Samsung,sum_device_mobile_brand_name_Xiaomi,sum_device_mobile_model_name_ChromeBook,sum_device_mobile_model_name_Safari,sum_device_mobile_model_name_iPad,sum_device_mobile_model_name_iPhone,sum_device_web_info_browser_AndroidWebview,sum_device_web_info_browser_Firefox,sum_device_web_info_browser_Safari,sum_geo_country_Canada,sum_geo_country_France,sum_geo_country_India,sum_geo_country_Other,sum_geo_country_Spain,sum_geo_country_UnitedKingdom,sum_traffic_source_medium_Other,sum_traffic_source_medium_cpc,sum_traffic_source_medium_organic,sum_traffic_source_medium_referral,sum_event_params_parent_page_CampusCollection,sum_event_params_parent_page_CheckoutConfirmation,sum_event_params_parent_page_EcoFriendly,sum_event_params_parent_page_Home,sum_event_params_parent_page_Lifestyle,sum_event_params_parent_page_New,sum_event_params_parent_page_Other,sum_event_params_parent_page_PaymentMethod,sum_event_params_parent_page_Sale,sum_event_params_parent_page_ShoppingCart,sum_event_params_child_page_Bags,sum_event_params_child_page_Google,sum_event_params_child_page_Hats,sum_event_params_child_page_Kids,sum_event_params_child_page_Notebooks,sum_event_params_child_page_Other,sum_event_params_child_page_Socks,sum_event_params_child_page_Stickers,sum_event_params_child_page_Womens,sum_event_params_child_page_Writing,sum_event_params_child_page_YouTube,sum_item_parent_category_Other,sum_item_parent_category_Stationery,sum_item_child_category_Drinkware,sum_item_child_category_Google,sum_item_child_category_MensUnisex,sum_item_child_category_SmallGoods,sum_item_child_subcategory_Backpacks,sum_item_child_subcategory_ElectronicsAccessories,sum_item_child_subcategory_Infant,sum_item_child_subcategory_MensTShirts,sum_item_child_subcategory_MugsTumblers,sum_item_child_subcategory_Other,sum_item_child_subcategory_WaterBottles,max_event_params_engagement_time_msec,max_item_promotions,avg_event_params_engagement_time_msec,avg_item_promotions,avg_item_quantity,avg_item_revenue_in_usd,pre_max_event_params_ga_session_number,pre_avg_event_params_engagement_time_msec,pre_sum_event_name_scroll,pre_sum_event_name_select_item,pre_sum_device_mobile_brand_name_Google,pre_sum_device_mobile_brand_name_Samsung,pre_sum_device_mobile_brand_name_Xiaomi,pre_sum_device_mobile_model_name_ChromeBook,pre_sum_device_mobile_model_name_Safari,pre_sum_device_mobile_model_name_iPad,pre_sum_device_web_info_browser_AndroidWebview,pre_sum_device_web_info_browser_Edge,pre_sum_device_web_info_browser_Firefox,pre_sum_geo_country_Canada,pre_sum_geo_country_India,pre_sum_geo_country_Other,pre_sum_geo_country_Spain,pre_sum_geo_country_UnitedKingdom,pre_sum_traffic_source_medium_Other,pre_sum_traffic_source_medium_cpc,pre_sum_traffic_source_medium_organic,pre_sum_traffic_source_medium_referral,pre_sum_event_params_parent_page_CampusCollection,pre_sum_event_params_parent_page_CheckoutConfirmation,pre_sum_event_params_parent_page_EcoFriendly,pre_sum_event_params_parent_page_Home,pre_sum_event_params_parent_page_PaymentMethod,pre_sum_event_params_parent_page_Sale,pre_sum_event_params_child_page_Hats,pre_sum_event_params_child_page_Socks,pre_sum_event_params_child_page_Stickers,pre_sum_event_params_child_page_Womens,pre_sum_event_params_child_page_YouTube,pre_sum_item_parent_category_New,pre_sum_item_child_category_Bags,pre_sum_item_child_category_Drinkware,pre_sum_item_child_category_Kids,pre_sum_item_child_category_MensUnisex,pre_sum_item_child_subcategory_Backpacks,pre_sum_item_child_subcategory_ElectronicsAccessories,pre_sum_item_child_subcategory_WaterBottles,days_first_session_to_transaction,his

In [ ]:
# create table schema to prevent errors when uploading data
schema = []

for col, dtype in zip(df_scaled.dtypes.index, df_scaled.dtypes.values):
    col_dict = {}
    col_dict['name'] = col

    if dtype == object:
        typ = 'STRING'
    elif dtype == bool:
        typ = 'BOOLEAN'
    elif dtype == 'datetime64[ns]':
        typ = 'DATETIME'
    elif dtype in (int, 'Int64', 'uint8'):
        typ = 'INTEGER'
    elif dtype == float:
        typ = 'FLOAT'
    else:
        print(dtype)
    col_dict['type'] = typ

    schema.append(col_dict)

In [ ]:
# df_scaled.to_gbq(f'{project_name}.return_prediction_ga4.step_6_merged_scaled',
#                     project_id=project_name,
#                     if_exists='replace',
#                     location=region,
#                     chunksize=100_000,
#                     table_schema=schema)

# Class imbalance
Another important check on your data that you should perform is checking for class imbalance. <br>
In this case we do have a continuous target, but we can artifically create classes of "return" or "no return" based on the target value.

In [ ]:
df['return_class'] = df.ecommerce_refund_value_in_usd.apply(lambda x: False if x == 0 else True)
df['return_class'].value_counts()

False    3434
True     1032
Name: return_class, dtype: int64

In [ ]:
df['return_class'].value_counts(normalize=True)

False   0.769
True    0.231
Name: return_class, dtype: float64

In this dataset we can see that the class imbalance between return and no return is not very severe. However, if your dataset is highly imbalanced you may want to rectify that imbalance using class balancing techniques such as over- and under-sampling or SMOTE.
**Add links etc**